In [31]:
# Librerias
import os
from bs4 import BeautifulSoup
from unidecode import unidecode
import datetime
import traceback
import json
import csv
import pandas as pd
import jellyfish
from collections import OrderedDict

In [32]:
# Parámetros
sourceDataFolder = "1-source-data"

bumeraniterationNumber = "4"
bumeranRootPath = sourceDataFolder + r"\main\bumeran\iteration-" + bumeraniterationNumber

linkediniterationNumber = "2"
linkedinRootPath = sourceDataFolder + r"\main\linkedin\iteration-" + linkediniterationNumber

intermFilesFolder = "2-intermediate-files"
mergedMainFolder = "3-merged-main"
logsFolder = "4-logs"
logFile = "10"

In [33]:
# Funciones utilitarias
def find_nth(haystack, needle, n):
    start = haystack.find(needle)
    while start >= 0 and n > 1:
        start = haystack.find(needle, start+len(needle))
        n -= 1
    return start

def parseLineBreaksAndAccents(text):
  return unidecode(" ".join(text.split()))

def parseNames(text):
  return unidecode(text).strip().title()

def findTags(tag, isBeforeChange):
  return tag.find("span", {"style": 'font-size:10.0pt;font-family:"Arial",sans-serif;mso-fareast-font-family:\n"Times New Roman";color:' + ('#2192C9' if isBeforeChange else '#0A26EE') })

def getChildIndex(mainChildTags, title, isBeforeChange):
  return next((index for index, tag in enumerate(mainChildTags) if ( findTags(tag, isBeforeChange).text == title if findTags(tag, isBeforeChange) else False )), None)

In [34]:
# Funciones principales
def getFiles(rootPath):
  startLimit = None
  topLimit = None
  folders = [f for f in os.listdir(rootPath) if os.path.isdir(os.path.join(rootPath, f))]
  files = []
  for folder in folders:
    folderFiles = [os.path.join(folder, f) for f in os.listdir(os.path.join(rootPath, folder)) if os.path.isfile(os.path.join(rootPath, folder, f))]
    files.extend(folderFiles)

  filterIsIn = [
    #r"analista-calidad\New application_ Analista de calidad - software  from Carlos Almeida Ignacio.html",
    #r"bandeja-entrada\New application_ Arquitecto de desarrollo TI from Rony Villanueva.html",
    #r"bandeja-entrada\New application_ Diseñador UX_UI from Estefania Pineyro.html",
    #r"analista-calidad\New application_ Analista de calidad - software  from Lizet Oria Rojas - ISTQB®.html",
    #r"analista-calidad\New application_ Analista de calidad - software  from Atenas Quintana Blas SFC™,SDC™,SMC™,CTFL.html",
    #r"analista-calidad\New application_ Analista de calidad from Yoshelin Ramos(2).html",
    #r"bandeja-entrada\New application_ .NET Developer from Wilder Jesus Ramirez Huamaccto.html",
    #r"analista-calidad\New application_ Analista de calidad from Hierson Armando Tineo(1).html",
    #r"bandeja-entrada\New application_ Desarrollador Java from Cesar Miguel Illesca Cangalaya.html",
    #r"analista-calidad\New application_ Analista de calidad from Juan Carlos Villagomez Canchari.html",
    #r"asistente-pmo\New application_ Asistente de PMO from Gerson Harold Trujillo Zuñiga.html",
    #r"analista-calidad\New application_ Analista de Calidad - TI from Roy Brandon Paul Ortiz Soto.html",
    #r"analista-funcional\New application_ Analista Funcional from Eloy Ivan Solano Coello.html",
    #r"ejecutivo-comercial\New application_ Ejecutivo Comercial - Sector TI from Gustavo Carquin mejia.html"
    #r"analista-funcional\New application_ Analista Funcional TI from Angie Pilar Chacaltana Cortez.html",
    #r"analista-funcional\New application_ Analista Funcional from Argenis Alexandra Moreno Rosales.html"
    #r"analista-calidad\Has recibido un CV para el aviso _Analista de Calidad Senior_(18).html",
    #r"analista-calidad\Has recibido un CV para el aviso _Analista de Calidad Senior_(24).html",
    #r"bandeja-entrada\Has recibido un CV para el aviso _Desarrollador.NET_(34).html",
    #r"bandeja-entrada\Has recibido un CV para el aviso _Ejecutivo(a) Comercial TI_(127).html"
    #r"bandeja-entrada\New application_ Cloud Specialist Jr. from Mario Sergio Campos Bernaola.html",
    #r"analista-funcional\New application_ Analista Funcional from Cyntia Caterine Huaytán Suazo.html",
  ]
  filesFilteredIsIn = [f for f in files if ( f in filterIsIn if filterIsIn else True )]

  # Casos particulares
  filterIsNotIn = [
    # Falta analizar los 19 casos particulares
  ]
  filesFilteredNotIn = [f for f in filesFilteredIsIn if ( f not in filterIsNotIn if filterIsNotIn else True )]

  filesPartitioned = filesFilteredNotIn[(startLimit-1 if startLimit else 0): (topLimit if topLimit else len(files))]
  return filesPartitioned


In [35]:
def getEncoding(file, stringLog, rootPath):
  encoding = "utf-8"
  try:
    with open(os.path.join(rootPath, file), "r") as f:
      encoding = 'windows-1252' if "charset=windows-1252" in f.read() else 'utf-16'
  except Exception as e:
    encoding = "utf-8"
    print(file)
    stringLog = stringLog + str(file) + "\n"
    traceback.print_exc()
    stringLog = stringLog + traceback.format_exc() + "\n"
    print()
    stringLog = stringLog + "\n"
    pass

  return encoding, stringLog


In [36]:
def getCandidateBumeran(file, index, encoding, stringLog):
  candidateData = {}
 
  try:
  # Abriendo el archivo
    with open(os.path.join(bumeranRootPath, file), "r", encoding=encoding) as myFile:
      #Parseando el archivo html a soup
      soup = BeautifulSoup(myFile.read(), "lxml")

      # Obteniendo cada uno de los campos para la data

      # Id del postulante
      #candidateData["id"] = str(index + 1)

      # Fecha de postulación
      rawPostulationDate = soup.find_all("span", {"style": "color:black"})[3].text
      postulationDate = datetime.datetime.strptime(rawPostulationDate, "%A, %B %d, %Y %I:%M %p")
      candidateData["postulationDate"] = postulationDate.strftime("%Y-%m-%d")

      # Definiendo los colores a usar para las búsquedas según las fechas
      isBeforeChange = True if (postulationDate <= datetime.datetime(2020,6,11,12,5,0)) else False
      # Linkedin comenzo a usarse en 22/04/25 23:06
      # El primer punto de inflexión es 22/09/23 02:15
      # El otro no tiene fecha definida, cambia muy poco el diseño pero agrega lo de adquirir más candidatos

      # Nombre del perfil
      candidateData["profileName"] = parseNames(parseLineBreaksAndAccents(soup.findAll("span", {"style": ("color:#008599" if isBeforeChange else "color:#E90066") })[0].text))

      # Nombre del postulante
      rawcandidateName = soup.find_all("ul", {"type": "disc"})[0].find_all("span")[0].text # El split join tambien quita saltos de línea
      candidateData["candidateName"] = parseNames(parseLineBreaksAndAccents(rawcandidateName))

      # Pais de residencia
      tagResidenceCountry = soup.find_all("ul", {"type": "disc"})[0].find_all("span")[2].text
      if tagResidenceCountry.find(",") == -1:
        candidateData["residenceCountry"] = ""
      else:
        rawResidenceCountry = tagResidenceCountry[0:tagResidenceCountry.find(",")]
        candidateData["residenceCountry"] = parseNames(parseLineBreaksAndAccents(rawResidenceCountry))

      # Canal
      candidateData["channel"] = "Bumeran"

      # Obteniendo todas las etiquetas principales
      mainDivTag = soup.find_all("div", {"style": 'background-position-x:50%;background-position-y:100%'})[0]
      mainChildTags = mainDivTag.find_all(recursive=False)

      # Experiencia laboral
      workExperienceIndex = getChildIndex(mainChildTags, "Experiencia laboral", isBeforeChange)
      educationIndex = getChildIndex(mainChildTags, "Educación", isBeforeChange)

      if workExperienceIndex and educationIndex:
        workExperienceTags = mainChildTags[workExperienceIndex+2:educationIndex]
        candidateData["lastWorkCenter"] = parseNames(parseLineBreaksAndAccents(workExperienceTags[0].find_all("b")[0].text))
        candidateData["lastWorkPosition"] = parseNames(parseLineBreaksAndAccents(workExperienceTags[1].find_all("b")[0].text))

        # Hay un caso que tiene span ya de por si en las experiencias laborales, el html 22, revisar
        daysOfExperience = 0
        for index in range(0, len(workExperienceTags), 2):
          startDate = datetime.datetime.strptime(parseLineBreaksAndAccents(workExperienceTags[index].text)[0: parseLineBreaksAndAccents(workExperienceTags[index].text).find(" - ")] , "%d-%m-%Y")
          endDateText = parseLineBreaksAndAccents(workExperienceTags[index].text)[parseLineBreaksAndAccents(workExperienceTags[index].text).find(" - ")+3: parseLineBreaksAndAccents(workExperienceTags[index].text).find(" | ")]
          endDate = datetime.datetime.strptime(endDateText, "%d-%m-%Y") if endDateText != "Presente" else datetime.datetime.strptime(rawPostulationDate, "%A, %B %d, %Y %I:%M %p")
          daysOfExperience = daysOfExperience + (endDate - startDate).days
        candidateData["yearsOfExperience"] = int(daysOfExperience/365)
        candidateData["worksNumber"] = int(len(workExperienceTags)/2)
      else:
        candidateData["lastWorkCenter"] = candidateData["lastWorkPosition"] = ""
        candidateData["yearsOfExperience"] = 0
        candidateData["worksNumber"] = 0

      # Carrera profesional (última alcanzada)
      educationIndex = getChildIndex(mainChildTags, "Educación", isBeforeChange)
      informaticsIndex = getChildIndex(mainChildTags, "Informática", isBeforeChange)

      if educationIndex and informaticsIndex:
        # Falta mejorar la validación (algunos tienen solo u, solo carrera o ninguno)
        educationTags = mainChildTags[educationIndex+2:informaticsIndex]

        boldTags0 = educationTags[0].find_all("b")
        if len(boldTags0) == 0:
          candidateData["studyCenter"] = ""
          candidateData["careerField"] = ""
        elif len(boldTags0) == 1:
          #print("GA")
          if (parseLineBreaksAndAccents(educationTags[0].text).find(",") - parseLineBreaksAndAccents(educationTags[0].text).find(" | ") == 3):
            candidateData["studyCenter"] = ""
            candidateData["careerField"] = parseNames(parseLineBreaksAndAccents(boldTags0[0].text.strip()))
          else:
            candidateData["studyCenter"] = parseNames(parseLineBreaksAndAccents(boldTags0[0].text.strip()))
            candidateData["careerField"] = ""
        else:
          #print("AG")
          #print(educationTags[0].find_all("b")[0])
          candidateData["studyCenter"] = parseNames(parseLineBreaksAndAccents(boldTags0[0].text.strip()))
          candidateData["careerField"] = parseNames(parseLineBreaksAndAccents(boldTags0[1].text.strip()))
        
        haveBoldTags1 = True if len(educationTags[1].find_all("b")) > 0 else False
        #if len(boldTags1) == 1:
        tempTags = educationTags[1].find_all("span")
        #print(tempTags[1:])
        statusDegreeIndex = next(((index+1 if haveBoldTags1 else index) for index, x in enumerate( tempTags[1:] if haveBoldTags1 else tempTags ) if "," in x.text), None)
        #print([x.text for index, x in enumerate(tempTags[1:])])
        #print(statusDegreeIndex)
        
        if statusDegreeIndex is not None:
          candidateData["careerStatus"] = parseNames(parseLineBreaksAndAccents(educationTags[1].find_all("span")[statusDegreeIndex].text.split(",")[1].strip()))
          candidateData["careerDegree"] = parseNames(parseLineBreaksAndAccents(educationTags[1].find_all("span")[statusDegreeIndex].text.split(",")[2].strip()[:-1]))
          candidateData["studiesNumber"] = int(len(educationTags)/2)
        else:
          candidateData["careerStatus"] = candidateData["careerDegree"] = ""
          candidateData["studiesNumber"] = 0
      else:
        candidateData["studyCenter"] = candidateData["careerField"] = candidateData["careerStatus"] = candidateData["careerDegree"] = ""
        candidateData["studiesNumber"] = 0
      #print(file)
      #print(candidateData["careerStatus"])
      #print(candidateData["careerDegree"])
      #print()

      # Habilidades técnicas
      informaticsIndex = getChildIndex(mainChildTags, "Informática", isBeforeChange)
      languageIndex = getChildIndex(mainChildTags, "Idiomas", isBeforeChange)

      if informaticsIndex and languageIndex:
        # Falta mejorar la validación (algunos tienen solo u, solo carrera o ninguno)
        technicalSkillsTags = mainChildTags[informaticsIndex+2:languageIndex][0].find_all("span")
        candidateData["technicalSkills"] = int(len(technicalSkillsTags)/4)

      else:
        candidateData["technicalSkills"] = 0

      # Falta habilidades blandas

      # Lenguajes
      languageIndex = getChildIndex(mainChildTags, "Idiomas", isBeforeChange)
      otherKnowledgesIndex = getChildIndex(mainChildTags, "Otros Conocimientos", isBeforeChange)

      if languageIndex and otherKnowledgesIndex:
        # Falta mejorar la validación (algunos tienen solo u, solo carrera o ninguno)
        languagesTags = mainChildTags[languageIndex+2:otherKnowledgesIndex][0].find_all("span")
        candidateData["languages"] = int(len(languagesTags)/7)

      else:
        candidateData["languages"] = 1

      # Otros conocimientos
      otherKnowledgesIndex = getChildIndex(mainChildTags, "Otros Conocimientos", isBeforeChange)
      endIndex = len(mainChildTags)-1

      if languageIndex and otherKnowledgesIndex:
        # Falta mejorar la validación (algunos tienen solo u, solo carrera o ninguno)
        otherKnowledgesTags = mainChildTags[otherKnowledgesIndex+2:endIndex][0].find_all("span")
        candidateData["anotherSkills"] = int(len(otherKnowledgesTags)/3)
      else:
        candidateData["anotherSkills"] = 0

      candidateData["references"] = 0

      # Salario pretendido
      tagsSalary = [index for index, tag in enumerate(soup.find_all("span")) if "Sueldo pretendido" in tag.text]
      if len(tagsSalary) > 0:
        rawSalary = soup.find_all("span")[tagsSalary[0]+1].text
        candidateData["salary"] = int(float(parseLineBreaksAndAccents(" ".join(rawSalary.split()))[1:].title()))
      else:
        candidateData["salary"] = 0

  except Exception as e:
    candidateData = {}
    print(file)
    stringLog = stringLog + str(file) + "\n"
    traceback.print_exc()
    stringLog = stringLog + traceback.format_exc() + "\n"
    print()
    stringLog = stringLog + "\n"
    pass

  return candidateData, stringLog


In [37]:
def getCandidateLinkedin(file, index, encoding, stringLog):
  candidateData = {}

  encoding = "utf-8"
  try:
    with open(os.path.join(linkedinRootPath, file), "r") as f:
      encoding = 'windows-1252' if "charset=windows-1252" in f.read() else 'utf-16'
  except Exception as e:
    encoding = "utf-8"
    print(file)
    stringLog = stringLog + str(file) + "\n"
    traceback.print_exc()
    stringLog = stringLog + traceback.format_exc() + "\n"
    print()
    stringLog = stringLog + "\n"
    pass

  try:
  # Abriendo el archivo
    with open(os.path.join(linkedinRootPath, file), "r", encoding=encoding) as myFile:
      #Parseando el archivo html a soup
      
      soup = BeautifulSoup(myFile.read(), "lxml")

      rawPostulationDate = soup.find_all("span", {"style": "color:black"})[3].text
      postulationDate = datetime.datetime.strptime(rawPostulationDate, "%A, %B %d, %Y %I:%M %p")

      rawProfileName = parseLineBreaksAndAccents(soup.find_all("span", {"style": "color:black"})[7].text)
      startIndexProfileName = rawProfileName.find(": ")
      endIndexProfileName = rawProfileName.find(" from ")

      if postulationDate <= datetime.datetime(2022,9,23,2,15,0):
        
        candidateData["postulationDate"] = postulationDate.strftime("%Y-%m-%d")

        candidateData["profileName"] = parseNames(parseLineBreaksAndAccents(rawProfileName[startIndexProfileName+1:endIndexProfileName]))

        candidateData["candidateName"] = parseNames(parseLineBreaksAndAccents(soup.find("span", {"style": "font-size:10.5pt;color:#262626;text-decoration:none;\n            text-underline:none"}).text))

        tempSpanTags = soup.find_all("span", {"style": 'font-size:9.0pt;font-family:\n            "Helvetica",sans-serif;color:#737373'})
        existsCountry = True if len(tempSpanTags) > 1 else False
        candidateData["residenceCountry"] = parseNames(parseLineBreaksAndAccents(tempSpanTags[len(tempSpanTags)-1].text)) if existsCountry else ""

        candidateData["channel"] = "Linkedin"

        tableTags = soup.find_all("table")

        notHaveScreening = True if "Screening qualifications" not in str(soup) else False
        startTag = 15 if notHaveScreening else 16
        endTag = len(tableTags)-3
        findedTableTags = tableTags[startTag:endTag]

        haveTags = [None, None, None, None, None]

        for index, tag in enumerate(findedTableTags):
          haveTags[0] = index if "Current experience" in tag.text else haveTags[0]
          haveTags[1] = index if "Past experience" in tag.text else haveTags[1]
          haveTags[2] = index if "Education" in tag.text else haveTags[2]
          haveTags[3] = index if "Skills matching your job" in tag.text else haveTags[3]
          haveTags[4] = index if "Highlight" in tag.text else haveTags[4]

        for index, haveTag in enumerate(haveTags):
          if index == 0:
            if haveTag is not None:
              #spanTags = findedTableTags[haveTag].find_all("tr")[1].find_all("span") Caso particular que no vale la pena
              #startSpanIndex = 0 if len(spanTags) == 2 else len(spanTags)-3
              candidateData["lastWorkCenter"] = parseNames(parseLineBreaksAndAccents(findedTableTags[haveTag].find_all("tr")[1].find_all("span")[0].text).split(" at ")[1])
              candidateData["lastWorkPosition"] = parseNames(parseLineBreaksAndAccents(findedTableTags[haveTag].find_all("tr")[1].find_all("span")[0].text).split(" at ")[0])
            else:
              candidateData["lastWorkCenter"] = candidateData["lastWorkPosition"] = ""
          
          elif index == 1:
            if haveTag is not None:
              daysOfExperience = 0
              trTag = findedTableTags[haveTag].find_all("tr")[1]
              pTags = trTag.find_all("p")
              for pTag in pTags:
                spanTag = parseLineBreaksAndAccents(pTag.find_all("span")[1].text)
                startDate = datetime.datetime.strptime(spanTag.split(" - ")[0], "%Y")
                endDate = datetime.datetime.strptime(spanTag.split(" - ")[1], "%Y")
                daysOfExperience = daysOfExperience + (endDate - startDate).days
              candidateData["yearsOfExperience"] = int(daysOfExperience/365)
              candidateData["worksNumber"] = int(len(pTags))
            else:
              candidateData["yearsOfExperience"] = 0
              candidateData["worksNumber"] = 0

          elif index == 2:
            # No se tiene el careerStatus en linkedin
            if haveTag is not None:
              trTags = findedTableTags[haveTag].find_all("tr")[1:]
              #print(trTags[0])
              firstCenterWithDateIndex = -1
              studiesNumber = 0
              for index, trTag in enumerate(trTags):
                if(len(trTag.find_all("span"))>1):
                  if(len(trTags[index-1].find_all("span")) == 1):
                    if firstCenterWithDateIndex == -1:
                      firstCenterWithDateIndex = index-1
                if(len(trTag.find_all("span"))==1):
                  studiesNumber = studiesNumber + 1
              
              #print(firstCenterWithDateIndex)
              #print(trTags[firstCenterWithDateIndex+1].find_all("span")[0].text.split(", ")[1])
              #print(trTags[firstCenterWithDateIndex+1].find_all("span")[0].text.split(", "))
              #print(parseLineBreaksAndAccents(trTags[firstCenterWithDateIndex+1].find_all("span")[0].text.split(", ")[1]))

              if firstCenterWithDateIndex != -1:
                candidateData["studyCenter"] = parseNames(parseLineBreaksAndAccents(trTags[firstCenterWithDateIndex].text))
                tempTagArray = trTags[firstCenterWithDateIndex+1].find_all("span")[0].text.split(",")
                candidateData["careerField"] = parseNames(parseLineBreaksAndAccents(trTags[firstCenterWithDateIndex+1].find_all("span")[0].text.split(",")[1])) if len(tempTagArray) > 1 else ""
                candidateData["careerStatus"] = "En Curso" if ("Present" in parseLineBreaksAndAccents(trTags[firstCenterWithDateIndex+1].find_all("span")[1].text)) else "Graduado"
                tempText = parseLineBreaksAndAccents(trTags[firstCenterWithDateIndex+1].find_all("span")[0].text.split(",")[0])
                candidateData["careerDegree"] = parseNames(tempText) if not tempText[0:4].isnumeric() else ""
              else:
                candidateData["studyCenter"] = parseNames(parseLineBreaksAndAccents(trTags[0].text))
                candidateData["careerField"] = ""
                candidateData["careerStatus"] = ""
                candidateData["careerDegree"] = ""
              
              candidateData["studiesNumber"] = studiesNumber
            else:
              candidateData["studyCenter"] = ""
              candidateData["careerField"] = ""
              candidateData["careerStatus"] = ""
              candidateData["careerDegree"] = ""
              candidateData["studiesNumber"] = 0
          elif index == 3:
            if haveTag is not None:
              skillsCounter = 0
              spanTags = findedTableTags[haveTag].find_all("tr")[1].find_all("span")
              for spanTag in spanTags:
                if "".join(spanTag["style"].split()) == 'font-size:10.5pt;font-family:"Helvetica",sans-serif;mso-fareast-font-family:"TimesNewRoman";color:#4D4D4D':
                  skillsCounter = skillsCounter + 1
              
              candidateData["technicalSkills"] = skillsCounter
            else:
              candidateData["technicalSkills"] = 0
            
            candidateData["languages"] = 1
            candidateData["anotherSkills"] = 0 if notHaveScreening else int(parseLineBreaksAndAccents(soup.find_all("table")[10].text)[parseLineBreaksAndAccents(soup.find_all("table")[10].text).find(": ")+2: parseLineBreaksAndAccents(soup.find_all("table")[10].text).find("/")])
          elif index == 4:
            if haveTag is not None:
              highlightText = parseLineBreaksAndAccents(findedTableTags[haveTag].find_all("tr")[1].text)
              isKnows = True if ("knows" in highlightText) else False
              #print(isKnows)
              isMoreThanTwo = True if ("others" in highlightText) else False
              if isKnows and isMoreThanTwo:
                candidateData["references"] = int(highlightText[highlightText.find(" and ")+5:highlightText.find(" others ")])
              elif isKnows and not isMoreThanTwo:
                candidateData["references"] = int(highlightText.count(" and ") + 1)
              else:
                candidateData["references"] = int(highlightText[highlightText.find(" hired ")+7:highlightText.find(" people ")])
            else:
              candidateData["references"] = 0
        candidateData["salary"] = 0
        
      else:
        candidateData["postulationDate"] = postulationDate.strftime("%Y-%m-%d")

        candidateData["profileName"] = parseNames(parseLineBreaksAndAccents(rawProfileName[startIndexProfileName+1:endIndexProfileName]))

        candidateData["candidateName"] = parseNames(parseLineBreaksAndAccents(soup.find("span", {"style": "color:#0A66C2;text-decoration:none;text-underline:none"}).text))
        
        tempSpanTags = soup.find_all("span", {"style": 'font-size:10.5pt;font-family:"Helvetica",sans-serif;\n          color:#737373'})
        existsCountry = True if len(tempSpanTags) > 1 else False
        candidateData["residenceCountry"] = parseNames(parseLineBreaksAndAccents(tempSpanTags[len(tempSpanTags)-1].text)) if existsCountry else ""

        candidateData["channel"] = "Linkedin"

        tableTags = soup.find_all("table")

        startTag = 14
        endTag = len(tableTags)-3
        findedTableTags = tableTags[startTag:endTag]

        haveTags = [None, None, None, None, None]

        for index, tag in enumerate(findedTableTags):
          haveTags[0] = index if "Current experience" in tag.text else haveTags[0]
          haveTags[1] = index if "Past experience" in tag.text else haveTags[1]
          haveTags[2] = index if "Education" in tag.text else haveTags[2]
          haveTags[3] = index if "Screening qualifications" in tag.text else haveTags[3]
          haveTags[4] = index if "Highlight" in tag.text else haveTags[4]
        
        for index, haveTag in enumerate(haveTags):
          if index == 0:
            if haveTag is not None:
              tempTrTags = parseLineBreaksAndAccents(findedTableTags[haveTag].find_all("tr")[1].find_all("span")[0].text).split(" at ")
              #print(tempTrTags[1].find_all("span")[0].text)
              tempWorkCenter = tempTrTags[1] if len(tempTrTags) > 1 else ""
              candidateData["lastWorkCenter"] = parseNames(tempWorkCenter[0:tempWorkCenter.rfind(" - ")-4]) if tempWorkCenter != "" else ""
              candidateData["lastWorkPosition"] = parseNames(parseLineBreaksAndAccents(findedTableTags[haveTag].find_all("tr")[1].find_all("span")[0].text).split(" at ")[0])
            else:
              candidateData["lastWorkCenter"] = candidateData["lastWorkPosition"] = ""
          
          elif index == 1:
            if haveTag is not None:
              daysOfExperience = 0
              trTags = [x for x in findedTableTags[haveTag].find_all("tr") if x.find("span", {"style": 'font-size:9.0pt;font-family:"Helvetica",sans-serif;\n          color:#737373'})]
              #print(trTags)
              for trTag in trTags:
                #print(pTag.text)
                spanTag = parseLineBreaksAndAccents(trTag.find_all("span")[0].text)
                #print(spanTag)
                #print("".isnumeric())
                if spanTag[spanTag.rfind(" - ")-4:spanTag.rfind(" - ")].isnumeric() and spanTag[spanTag.rfind(" - ")+3:spanTag.rfind(" - ")+7].isnumeric():
                  startDate = datetime.datetime.strptime(spanTag[spanTag.rfind(" - ")-4:spanTag.rfind(" - ")], "%Y")
                  endDate = datetime.datetime.strptime(spanTag[spanTag.rfind(" - ")+3:spanTag.rfind(" - ")+7], "%Y")
                  daysOfExperience = daysOfExperience + (endDate - startDate).days
              candidateData["yearsOfExperience"] = int(daysOfExperience/365)
              candidateData["worksNumber"] = int(len(trTags))
            else:
              candidateData["yearsOfExperience"] = 0
              candidateData["worksNumber"] = 0

          elif index == 2:
            # No se tiene el careerStatus en linkedin
            if haveTag is not None:
              trTags = findedTableTags[haveTag].find_all("tr")[1:]
              #print(trTags[0])
              firstCenterWithDateIndex = -1
              studiesNumber = 0
              for index, trTag in enumerate(trTags):
                if " - " in parseLineBreaksAndAccents(trTag.text):
                  if " - " not in parseLineBreaksAndAccents(trTags[index-1].text):
                    if firstCenterWithDateIndex == -1:
                      firstCenterWithDateIndex = index-1
                if(" - " not in parseLineBreaksAndAccents(trTag.text)):
                  studiesNumber = studiesNumber + 1
              #print(firstCenterWithDateIndex)
              if firstCenterWithDateIndex != -1:
                #print("GA")
                candidateData["studyCenter"] = parseNames(parseLineBreaksAndAccents(trTags[firstCenterWithDateIndex].text))
                tempTagArray = parseNames(parseLineBreaksAndAccents(trTags[firstCenterWithDateIndex+1].find("span").text))
                candidateData["careerField"] = parseNames(tempTagArray[tempTagArray.find(", ")+1:tempTagArray.rfind(" - ")-4])
                candidateData["careerStatus"] = "En Curso" if ("Present" in tempTagArray) else "Graduado"
                candidateData["careerDegree"] = parseNames(tempTagArray[0:tempTagArray.find(", ")]) if tempTagArray.find(", ") != -1 else ""
                #print(candidateData)
              else:
                candidateData["studyCenter"] = parseNames(parseLineBreaksAndAccents(trTags[0].text))
                candidateData["careerField"] = ""
                candidateData["careerStatus"] = ""
                candidateData["careerDegree"] = ""
              
              candidateData["studiesNumber"] = studiesNumber
            else:
              candidateData["studyCenter"] = ""
              candidateData["careerField"] = ""
              candidateData["careerStatus"] = ""
              candidateData["careerDegree"] = ""
              candidateData["studiesNumber"] = 0
          
          elif index == 3:
            if haveTag is not None:
              candidateData["technicalSkills"] = 0
              candidateData["languages"] = 1
              candidateData["anotherSkills"] = int(parseLineBreaksAndAccents(findedTableTags[haveTag].find_all("tr")[1].text)[0])
            else:
              candidateData["technicalSkills"] = 0
              candidateData["languages"] = 1
              candidateData["anotherSkills"] = 0
            
          elif index == 4:
            if haveTag is not None:
              highlightText = parseLineBreaksAndAccents(findedTableTags[haveTag].find_all("tr")[1].text)
              #print(highlightText)
              isKnows = True if ("knows" in highlightText) else False
              #print(isKnows)
              isMoreThanTwo = True if ("others" in highlightText) else False
              if isKnows and isMoreThanTwo:
                candidateData["references"] = int(highlightText[highlightText.find(" and ")+5:highlightText.find(" others ")])
              elif isKnows and not isMoreThanTwo:
                candidateData["references"] = int(highlightText.count(" and ") + 1)
              else:
                candidateData["references"] = int(highlightText[highlightText.find(" hired ")+7:highlightText.find(" people ")])
            else:
              candidateData["references"] = 0
        candidateData["salary"] = 0
      
  except Exception as e:

    candidateData = {}
    print(file)
    stringLog = stringLog + str(file) + "\n"
    traceback.print_exc()
    stringLog = stringLog + traceback.format_exc() + "\n"
    print()
    stringLog = stringLog + "\n"
    pass
  
  return candidateData, stringLog


In [38]:
def readJson(path, encoding='utf-8', errors=None):
  with open (path, "r", encoding=encoding, errors=errors) as file:
    data = json.loads(file.read())
  return data


In [39]:
def writeJson(data, pathJson, pathCsv, encoding='utf-8'):
  with open(pathJson, 'w', encoding=encoding) as file:
    json.dump(data, file, indent=4, ensure_ascii=False)

In [40]:
def writeCsv(data, pathCsv, encoding='utf-8'):
  with open(pathCsv, 'w', newline='', encoding=encoding) as file:
    if data:
      writer = csv.DictWriter(file, fieldnames=data[0].keys(), lineterminator='\n')
      writer.writeheader()
      writer.writerows(data)
    else:
      file.write("")


In [41]:
def iterateFiles(files, source, rootPath):
  stringLog = ""

  # Definiendo la data final a recoger
  data = []
  # Iterando por cada postulación
  for index, file in enumerate(files):
    # Obteniendo el encoding
    encoding, stringLog = getEncoding(file, stringLog, rootPath)

    candidate, stringLog = getCandidateBumeran(file, index, encoding, stringLog) if source == 'bumeran' else getCandidateLinkedin(file, index, encoding, stringLog)
    
    if (candidate):
      data.append(candidate)

  return data, stringLog

In [42]:
def readAndWriteMain(source):
  rootPath = bumeranRootPath if source == "bumeran" else linkedinRootPath
  files = getFiles(rootPath)
  # Iterando sobre los archivos y calculando la data
  data, stringLog = iterateFiles(files, source, rootPath)

  # Escribiendo la data y el log
  with open(os.path.join(intermFilesFolder, source + '.json'), 'w') as f:
    json.dump(data, f, indent=4, ensure_ascii=False)
  
  with open(os.path.join(logsFolder, logFile + "-" + source + ".txt"), "w", encoding='utf-16') as f:
    f.write(stringLog)

  return data


In [43]:
def readAndWriteMergedMain(mainData):
  mainMergeddata = []
  for elem in mainData:
    mainMergeddata.extend(elem)

  writeJson(mainMergeddata, os.path.join(mergedMainFolder, "result.json"), 'utf-8')
  writeCsv(mainMergeddata, os.path.join(mergedMainFolder, "result.csv"), 'utf-8')

  return mainMergeddata

In [44]:
def main():
  startTime = datetime.datetime.now()
  print("Inicio: " + str(startTime))
  print("Se inició el procesamiento")

  # El dataset principal que sea el de bumeran
  # Los otros archivos que solo se usen para actualizar la variable objetivo
  # Esto porque los otros orígenes están incompletos y costaría llenar los campos (incluso el excel que se hizo)
  isLoadedBumeran = False
  isLoadedLinkedin = False
  isMergedMain = False
  #False

  # Leyendo archivos de entrada
  bumeranData = readJson(os.path.join(intermFilesFolder, 'bumeran.json')) if isLoadedBumeran else readAndWriteMain('bumeran')
  print("Se terminó de procesar Bumeran")

  linkedinData = readJson(os.path.join(intermFilesFolder, 'linkedin.json')) if isLoadedLinkedin else readAndWriteMain('linkedin')
  print("Se terminó de procesar Linkedin")

  # Uniendo la data principal (bumeran + linkedin)
  mergedMainData = readJson(os.path.join(mergedMainFolder, 'result.json')) if isMergedMain else readAndWriteMergedMain([bumeranData, linkedinData])
  print("Se terminó de unir la data principal")
  
  endTime = datetime.datetime.now()

  print("Fin: " + str(endTime))
  print("Tiempo: " + str(endTime-startTime))

if __name__ == "__main__":
  main()


Inicio: 2023-05-10 01:40:50.747398
Se inició el procesamiento
analista-calidad\Has recibido un CV para el aviso _Analista de Calidad_(76).html

analista-calidad\Has recibido un CV para el aviso _Analista de Calidad_(76).html



Traceback (most recent call last):
  File "C:\Users\Ronaldo\AppData\Local\Temp\ipykernel_5304\3301910123.py", line 5, in getEncoding
    encoding = 'windows-1252' if "charset=windows-1252" in f.read() else 'utf-16'
  File "c:\Users\Ronaldo\AppData\Local\Programs\Python\Python39\lib\encodings\cp1252.py", line 23, in decode
    return codecs.charmap_decode(input,self.errors,decoding_table)[0]
UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 126888: character maps to <undefined>
Traceback (most recent call last):
  File "C:\Users\Ronaldo\AppData\Local\Temp\ipykernel_5304\2750435797.py", line 8, in getCandidateBumeran
    soup = BeautifulSoup(myFile.read(), "lxml")
  File "c:\Users\Ronaldo\AppData\Local\Programs\Python\Python39\lib\codecs.py", line 322, in decode
    (result, consumed) = self._buffer_decode(data, self.errors, final)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte


analista-calidad\Has recibido un CV para el aviso _Analista QA_(29).html

analista-calidad\Has recibido un CV para el aviso _Analista QA_(29).html



Traceback (most recent call last):
  File "C:\Users\Ronaldo\AppData\Local\Temp\ipykernel_5304\3301910123.py", line 5, in getEncoding
    encoding = 'windows-1252' if "charset=windows-1252" in f.read() else 'utf-16'
  File "c:\Users\Ronaldo\AppData\Local\Programs\Python\Python39\lib\encodings\cp1252.py", line 23, in decode
    return codecs.charmap_decode(input,self.errors,decoding_table)[0]
UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 123724: character maps to <undefined>
Traceback (most recent call last):
  File "C:\Users\Ronaldo\AppData\Local\Temp\ipykernel_5304\2750435797.py", line 8, in getCandidateBumeran
    soup = BeautifulSoup(myFile.read(), "lxml")
  File "c:\Users\Ronaldo\AppData\Local\Programs\Python\Python39\lib\codecs.py", line 322, in decode
    (result, consumed) = self._buffer_decode(data, self.errors, final)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte


analista-funcional\Has recibido un CV para el aviso _Analista Funcional_(110).html



...

bandeja-entrada\Your job post is live!(1).html

bandeja-entrada\Your job post is live!(10).html

bandeja-entrada\Your job post is live!(11).html

bandeja-entrada\Your job post is live!(2).html

bandeja-entrada\Your job post is live!(3).html

bandeja-entrada\Your job post is live!(4).html

bandeja-entrada\Your job post is live!(5).html

bandeja-entrada\Your job post is live!(6).html



Traceback (most recent call last):
  File "C:\Users\Ronaldo\AppData\Local\Temp\ipykernel_5304\3591020669.py", line 38, in getCandidateLinkedin
    candidateData["candidateName"] = parseNames(parseLineBreaksAndAccents(soup.find("span", {"style": "font-size:10.5pt;color:#262626;text-decoration:none;\n            text-underline:none"}).text))
AttributeError: 'NoneType' object has no attribute 'text'
Traceback (most recent call last):
  File "C:\Users\Ronaldo\AppData\Local\Temp\ipykernel_5304\3591020669.py", line 38, in getCandidateLinkedin
    candidateData["candidateName"] = parseNames(parseLineBreaksAndAccents(soup.find("span", {"style": "font-size:10.5pt;color:#262626;text-decoration:none;\n            text-underline:none"}).text))
AttributeError: 'NoneType' object has no attribute 'text'
Traceback (most recent call last):
  File "C:\Users\Ronaldo\AppData\Local\Temp\ipykernel_5304\3591020669.py", line 38, in getCandidateLinkedin
    candidateData["candidateName"] = parseNames(parseLine

bandeja-entrada\Your job post is live!(7).html

bandeja-entrada\Your job post is live!(8).html

bandeja-entrada\Your job post is live!(9).html

bandeja-entrada\Your job post is live!.html



Traceback (most recent call last):
  File "C:\Users\Ronaldo\AppData\Local\Temp\ipykernel_5304\3591020669.py", line 38, in getCandidateLinkedin
    candidateData["candidateName"] = parseNames(parseLineBreaksAndAccents(soup.find("span", {"style": "font-size:10.5pt;color:#262626;text-decoration:none;\n            text-underline:none"}).text))
AttributeError: 'NoneType' object has no attribute 'text'
Traceback (most recent call last):
  File "C:\Users\Ronaldo\AppData\Local\Temp\ipykernel_5304\3591020669.py", line 38, in getCandidateLinkedin
    candidateData["candidateName"] = parseNames(parseLineBreaksAndAccents(soup.find("span", {"style": "font-size:10.5pt;color:#262626;text-decoration:none;\n            text-underline:none"}).text))
AttributeError: 'NoneType' object has no attribute 'text'
Traceback (most recent call last):
  File "C:\Users\Ronaldo\AppData\Local\Temp\ipykernel_5304\3591020669.py", line 38, in getCandidateLinkedin
    candidateData["candidateName"] = parseNames(parseLine

Se terminó de procesar Linkedin
Se terminó de unir la data principal
Fin: 2023-05-10 01:44:02.253971
Tiempo: 0:03:11.506573
